### Start the Cluster

In [60]:
!gcloud container clusters create microblog-cluster --num-nodes=2

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Creating cluster microblog-cluster in us-central1-a...done.
Created [https://container.googleapis.com/v1/projects/going-tfx/zones/us-central1-a/clusters/microblog-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-a/microblog-cluster?project=going-tfx
kubeconfig entry generated for microblog-cluster.
NAME               LOCATION       MASTER_VERSION  MASTER_IP      MACHINE_TYPE   NODE_VERSION  NUM_NODES  STATUS
microblog-cluster  us-central1-a  1.10.9-gke.5    35.239.217.33  n1-standard-1  1.10.9-gke.5  2          RUNNING


In [61]:
!gcloud container clusters get-credentials microblog-cluster --zone us-central1-a --project going-tfx

Fetching cluster endpoint and auth data.
kubeconfig entry generated for microblog-cluster.


---
### DNS Records

In [20]:
!gcloud compute addresses list

NAME             ADDRESS/RANGE   TYPE  PURPOSE  NETWORK  REGION       SUBNET  STATUS
hello-server-ip  35.186.195.159                                               RESERVED
jenkins          35.244.196.48                                                RESERVED
microblog        35.244.236.101                                               RESERVED
jenkins          35.184.14.235                           us-central1          IN_USE


In [21]:
!gcloud dns record-sets list --zone=fnf-ninja-on-going-tfx

NAME                  TYPE  TTL    DATA
fnf.ninja.            NS    21600  ns-cloud-c1.googledomains.com.,ns-cloud-c2.googledomains.com.,ns-cloud-c3.googledomains.com.,ns-cloud-c4.googledomains.com.
fnf.ninja.            SOA   21600  ns-cloud-c1.googledomains.com. cloud-dns-hostmaster.google.com. 1 21600 3600 259200 300
microblog.fnf.ninja.  A     300    35.244.236.101


In [55]:
!gcloud dns record-sets transaction start --zone=fnf-ninja-on-going-tfx

Transaction started [transaction.yaml].


In [56]:
!gcloud dns record-sets transaction add --zone=fnf-ninja-on-going-tfx --name="microblog.fnf.ninja"\
    --type=A --ttl=300 "35.244.236.101"

Record addition appended to transaction at [transaction.yaml].


In [57]:
!gcloud dns record-sets transaction execute --zone=fnf-ninja-on-going-tfx

ERROR: (gcloud.dns.record-sets.transaction.execute) HTTPError 409: The resource 'entity.change.additions[1]' named 'microblog.fnf.ninja. (A)' already exists


---
### Deploy and serve the application

In [62]:
!kubectl run microblog --image=us.gcr.io/going-tfx/microblog:v1 --port=5000

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
deployment.apps/microblog created


In [63]:
!kubectl apply -f k8s/service.yaml

service/microblog created


In [64]:
!kubectl apply -f k8s/ingress.yaml

ingress.extensions/microblog-ingress created


In [65]:
!kubectl describe service microblog

Name:                     microblog
Namespace:                default
Labels:                   run=microblog
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{"service.alpha.kubernetes.io/app-protocols":"{\"my-https-port\":\"HTTPS\"}"...
                          service.alpha.kubernetes.io/app-protocols: {"my-https-port":"HTTPS"}
Selector:                 run=microblog
Type:                     NodePort
IP:                       10.19.246.11
Port:                     <unset>  5000/TCP
TargetPort:               5000/TCP
NodePort:                 <unset>  31709/TCP
Endpoints:                10.16.0.8:5000
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [66]:
!kubectl describe -f k8s/ingress.yaml

Name:             microblog-ingress
Namespace:        default
Address:          
Default backend:  default-http-backend:80 (10.16.0.7:8080)
Rules:
  Host  Path  Backends
  ----  ----  --------
  *     
        /   microblog:5000 (<none>)
Annotations:
  kubernetes.io/ingress.global-static-ip-name:       microblog
  certmanager.k8s.io/cluster-issuer:                 letsencrypt-prod
  kubectl.kubernetes.io/last-applied-configuration:  {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"certmanager.k8s.io/cluster-issuer":"letsencrypt-prod","kubernetes.io/ingress.global-static-ip-name":"microblog"},"labels":{"app":"microblog"},"name":"microblog-ingress","namespace":"default"},"spec":{"rules":[{"http":{"paths":[{"backend":{"serviceName":"microblog","servicePort":5000},"path":"/"}]}}]}}

Events:
  Type    Reason  Age   From                     Message
  ----    ------  ----  ----                     -------
  Normal  ADD     9s    loadbalancer-controller  default/

---
#### Check if the service can be located through the ingress
Give it a couple of minutes, or check on the Kubernetes console for completion

In [71]:
!curl http://microblog.fnf.ninja | grep Wolfie

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1621  100  1621    0     0  95849      0 --:--:-- --:--:-- --:--:--   98k
<h1>Hello, Wolfie!</h1>


#### Remove it, it will be replaced by a proper TLS ingress

In [72]:
!kubectl delete -f k8s/ingress.yaml

ingress.extensions "microblog-ingress" deleted


---
### Add certificate infrastructure via helm

In [73]:
!kubectl create serviceaccount -n kube-system tiller

serviceaccount/tiller created


In [74]:
!kubectl create clusterrolebinding tiller-binding --clusterrole=cluster-admin --serviceaccount kube-system:tiller

clusterrolebinding.rbac.authorization.k8s.io/tiller-binding created


In [75]:
!helm init --service-account tiller

$HELM_HOME has been configured at /home/wgiersche/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [76]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


---
### Install cert manager

In [77]:
!helm install --name cert-manager --version v0.3.1 --namespace kube-system stable/cert-manager

NAME:   cert-manager
LAST DEPLOYED: Sat Dec 15 16:27:53 2018
NAMESPACE: kube-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                       SECRETS  AGE
cert-manager-cert-manager  1        0s

==> v1beta1/CustomResourceDefinition
NAME                               AGE
certificates.certmanager.k8s.io    0s
clusterissuers.certmanager.k8s.io  0s
issuers.certmanager.k8s.io         0s

==> v1beta1/ClusterRole
NAME                       AGE
cert-manager-cert-manager  0s

==> v1beta1/ClusterRoleBinding
NAME                       AGE
cert-manager-cert-manager  0s

==> v1beta1/Deployment
NAME                       DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
cert-manager-cert-manager  1        1        1           0          0s

==> v1/Pod(related)
NAME                                       READY  STATUS             RESTARTS  AGE
cert-manager-cert-manager-698758c6d-zcn7c  0/1    ContainerCreating  0         0s


NOTES:
cert-manager has been deployed successfully!

In order

In [78]:
!kubectl get deployment -n kube-system cert-manager-cert-manager

NAME                        DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
cert-manager-cert-manager   1         1         1            1           6s


In [79]:
!kubectl apply -f k8s/clusterissuer.yaml

clusterissuer.certmanager.k8s.io/letsencrypt-staging created
clusterissuer.certmanager.k8s.io/letsencrypt-prod created


In [80]:
!kubectl describe -f k8s/clusterissuer.yaml | grep Name:

Name:         letsencrypt-staging
  Cluster Name:        
      Name:  letsencrypt-staging
Name:         letsencrypt-prod
  Cluster Name:        
      Name:  letsencrypt-prod


In [81]:
!kubectl apply -f k8s/certificate.yaml

certificate.certmanager.k8s.io/microblog-fnf-ninja-tls created


In [97]:
!kubectl get pods -n kube-system | grep cert-manager
pod = !kubectl get pods -n kube-system | grep cert-manager
pod = pod[0].split(' ')[0]

cert-manager-cert-manager-698758c6d-zcn7c                     1/1     Running   0          6m


In [98]:
!kubectl logs -n kube-system $pod | tail -40

E1215 16:31:39.122754       1 controller.go:186] certificates controller: Re-queuing item "default/microblog-fnf-ninja-tls" due to error processing: ingress.extensions "microblog-ingress-tls" not found
I1215 16:32:39.123096       1 controller.go:177] certificates controller: syncing item 'default/microblog-fnf-ninja-tls'
I1215 16:32:39.123202       1 sync.go:239] Preparing certificate default/microblog-fnf-ninja-tls with issuer
I1215 16:32:39.123210       1 acme.go:159] getting private key (letsencrypt-prod->tls.key) for acme issuer kube-system/letsencrypt-prod
I1215 16:32:39.123634       1 logger.go:27] Calling GetOrder
I1215 16:32:39.292226       1 logger.go:52] Calling GetAuthorization
I1215 16:32:39.361989       1 logger.go:72] Calling HTTP01ChallengeResponse
I1215 16:32:39.362035       1 prepare.go:263] Cleaning up old/expired challenges for Certificate default/microblog-fnf-ninja-tls
I1215 16:32:39.362050       1 logger.go:47] Calling GetChallenge
I1215 16:32:40.025768       1 ht

In [99]:
!kubectl describe -f k8s/certificate.yaml

Name:         microblog-fnf-ninja-tls
Namespace:    default
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"certmanager.k8s.io/v1alpha1","kind":"Certificate","metadata":{"annotations":{},"name":"microblog-fnf-ninja-tls","namespace"...
API Version:  certmanager.k8s.io/v1alpha1
Kind:         Certificate
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-12-15T16:28:29Z
  Generation:          1
  Resource Version:    2711
  Self Link:           /apis/certmanager.k8s.io/v1alpha1/namespaces/default/certificates/microblog-fnf-ninja-tls
  UID:                 749691e7-0086-11e9-a573-42010a800190
Spec:
  Acme:
    Config:
      Domains:
        microblog.fnf.ninja
      Http 01:
        Ingress:  microblog-ingress-tls
  Common Name:    microblog.fnf.ninja
  Dns Names:
    microblog.fnf.ninja
  Issuer Ref:
    Kind:       ClusterIssuer
    Name:       letsencrypt-prod
  Secret Name:  microblog-fnf-ninja-tls
Status:
 

In [104]:
!kubectl delete -f k8s/ingress-tls.yaml

ingress.extensions "microblog-ingress-tls" deleted


In [105]:
!kubectl apply -f k8s/ingress-tls.yaml

ingress.extensions/microblog-ingress-tls created


In [106]:
!kubectl get secrets

NAME                  TYPE                                  DATA   AGE
default-token-lnbmc   kubernetes.io/service-account-token   3      41m
microblog-fnf-ninja   kubernetes.io/tls                     2      5m


In [112]:
!curl https://microblog.fnf.ninja | grep Wolfie

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1621  100  1621    0     0  57256      0 --:--:-- --:--:-- --:--:-- 57892
<h1>Hello, Wolfie!</h1>
